![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&dt=BQML+Demo+2024.ipynb)

# BQML DEMO 2024

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/TensorFlow/TensorFlow%20Basics%20-%20Data%20To%20Training.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('bigframes', 'bigframes'),,
    ('google.cloud.bigquery_connection_v1', 'google-cloud-bigquery-connection')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

In [4]:
#!sudo apt-get -qq install graphviz

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
# BigQuery Parameters
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'bqml_2024'
BQ_TABLE_PREFIX = 'thelook'
BQ_REGION = 'us'
BQ_SOURCE_DATASET = 'bigquery-public-data.thelook_ecommerce'

packages:

In [69]:
from google.cloud import bigquery
from google.cloud import bigquery_connection_v1 as bq_connection
import bigframes.pandas as bpd
import pandas as pd
import numpy as np

clients:

In [9]:
bq = bigquery.Client(project = PROJECT_ID)
bpd.options.bigquery.project = PROJECT_ID

---
## Environment Setup

### BigQuery Dataset

In [70]:
try:
    ds = bq.get_dataset(f'{BQ_PROJECT}.{BQ_DATASET}')
    print('Found the BigQuery Dataset: ', ds.full_dataset_id)
except:
    ds = bigquery.DatasetReference(BQ_PROJECT, BQ_DATASET)
    ds.location = BQ_REGION
    ds = bq.create_dataset(dataset = ds, exists_ok = True)
    print('Created the BigQuery Dataset: ', ds.full_dataset_id)

Found the BigQuery Dataset:  statmike-mlops-349915:bqml_2024


### Connection Requirement

To make a remote connection using BigQuery ML, BigQuery uses a CLOUD_RESOURCE connection. [Reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model#connection)

Create a new connection with type `CLOUD_RESOURCE`: First, check for existing connection.

In [72]:
try:
    response = bq_connection.ConnectionServiceClient().get_connection(
            request = bq_connection.GetConnectionRequest(
                name = f"projects/{BQ_PROJECT}/locations/{BQ_REGION}/connections/{BQ_DATASET}"
            )
    )
    print(f'Found existing connection with service account: {response.cloud_resource.service_account_id}')
    service_account = response.cloud_resource.service_account_id
except Exception:
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": f"projects/{BQ_PROJECT}/locations/{BQ_REGION}",
            "connection_id": f"{BQ_DATASET}",
            "connection": bq_connection.types.Connection(
                {
                    "friendly_name": f"{BQ_DATASET}",
                    "cloud_resource": bq_connection.CloudResourceProperties({})
                }
            )
        }
    )
    response = bq_connection.ConnectionServiceClient().create_connection(request)
    print(f'Created new connection with service account: {response.cloud_resource.service_account_id}')
    service_account = response.cloud_resource.service_account_id

Created new connection with service account: bqcx-1026793852137-d2h9@gcp-sa-bigquery-condel.iam.gserviceaccount.com


Assign the service account the Vertex AI User role:

In [73]:
# for llm usage on vertex
!gcloud projects add-iam-policy-binding {BQ_PROJECT} --member=serviceAccount:{service_account} --role=roles/aiplatform.user

Updated IAM policy for project [statmike-mlops-349915].
bindings:
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-1026793852137-bmph@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-d2h9@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-dyw1@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-pdxa@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-te86@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-tqpc@gcp-sa-big

In [176]:
# for cloud nlp usage:
!gcloud projects add-iam-policy-binding {BQ_PROJECT} --member=serviceAccount:{service_account} --role=roles/serviceusage.serviceUsageConsumer

Updated IAM policy for project [statmike-mlops-349915].
bindings:
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-1026793852137-bmph@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-d2h9@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-dyw1@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-pdxa@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-te86@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-tqpc@gcp-sa-big

In [187]:
# for cloud translate usage:
!gcloud projects add-iam-policy-binding {BQ_PROJECT} --member=serviceAccount:{service_account} --role=roles/cloudtranslate.user

Updated IAM policy for project [statmike-mlops-349915].
bindings:
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-1026793852137-bmph@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-d2h9@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-dyw1@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-pdxa@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-te86@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - serviceAccount:bqcx-1026793852137-tqpc@gcp-sa-big

### Create The Remote Models In BigQuery

In [78]:
# Create Remote Model In BigQuery
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.VERTEX_LLM`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(ENDPOINT = 'text-bison@002')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [79]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.VERTEX_LLM`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(ENDPOINT = 'text-bison@002')



In [80]:
# Create Remote Model In BigQuery
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_TRANSLATE`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_TRANSLATE_V3')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [81]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.CLOUD_TRANSLATE`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_TRANSLATE_V3')



In [82]:
# Create Remote Model In BigQuery
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_NLP`
    REMOTE WITH CONNECTION `{BQ_PROJECT}.{BQ_REGION}.{BQ_DATASET}`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_NATURAL_LANGUAGE_V1')
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

In [83]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_2024.CLOUD_NLP`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.bqml_2024`
    OPTIONS(REMOTE_SERVICE_TYPE = 'CLOUD_AI_NATURAL_LANGUAGE_V1')



---
## Intro & Explore

In [177]:
query = f'''
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts` AS
WITH
    TOP5 AS (
        SELECT product_id, count(*) as total
        FROM `{BQ_SOURCE_DATASET}.order_items`
        GROUP BY product_id
    )
SELECT
    t.product_id,
    t.total,
    p.category,
    p.department,
    p.name
FROM TOP5 as t
JOIN `{BQ_SOURCE_DATASET}.products` as p on p.id = t.product_id
WHERE p.department = 'Men'
ORDER by t.total DESC
LIMIT 5
'''
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_topMensProducts` AS
WITH
    TOP5 AS (
        SELECT product_id, count(*) as total
        FROM `bigquery-public-data.thelook_ecommerce.order_items`
        GROUP BY product_id
    )
SELECT
    t.product_id,
    t.total,
    p.category,
    p.department,
    p.name
FROM TOP5 as t
JOIN `bigquery-public-data.thelook_ecommerce.products` as p on p.id = t.product_id
WHERE p.department = 'Men'
ORDER by t.total DESC
LIMIT 5



In [178]:
job = bq.query(query = query)
job.result()

In [180]:
query = f'''
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts` AS
SELECT
    * EXCEPT(ml_generate_text_llm_result, ml_generate_text_status),
    ml_generate_text_llm_result as description
FROM ML.GENERATE_TEXT(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.VERTEX_LLM`,
    (SELECT *,
        CONCAT(
            'Create a descriptive paragraph of no more than 25 words for a product with in a department named ', department,
            ', category named "', category, '"',
            'and the following description: ', name
        ) as prompt
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts`),
    STRUCT(200 AS max_output_tokens, TRUE AS flatten_json_output)
)
'''
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_topMensProducts` AS
SELECT
    * EXCEPT(ml_generate_text_llm_result, ml_generate_text_status),
    ml_generate_text_llm_result as description
FROM ML.GENERATE_TEXT(
    MODEL `statmike-mlops-349915.bqml_2024.VERTEX_LLM`,
    (SELECT *,
        CONCAT(
            'Create a descriptive paragraph of no more than 25 words for a product with in a department named ', department,
            ', category named "', category, '"',
            'and the following description: ', name
        ) as prompt
        FROM `statmike-mlops-349915.bqml_2024.thelook_topMensProducts`),
    STRUCT(200 AS max_output_tokens, TRUE AS flatten_json_output)
)



In [181]:
job = bq.query(query = query)
job.result()

In [183]:
query = f'''
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts` AS
SELECT
    * EXCEPT(text_content, ml_understand_text_result, ml_understand_text_status),
    ml_understand_text_result.document_sentiment.score as sentiment_score
FROM ML.UNDERSTAND_TEXT(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_NLP`,
    (SELECT *, description as text_content
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts`),
    STRUCT('ANALYZE_SENTIMENT' AS nlu_option)
)
'''
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_topMensProducts` AS
SELECT
    * EXCEPT(text_content, ml_understand_text_result, ml_understand_text_status),
    ml_understand_text_result.document_sentiment.score as sentiment_score
FROM ML.UNDERSTAND_TEXT(
    MODEL `statmike-mlops-349915.bqml_2024.CLOUD_NLP`,
    (SELECT *, description as text_content
        FROM `statmike-mlops-349915.bqml_2024.thelook_topMensProducts`),
    STRUCT('ANALYZE_SENTIMENT' AS nlu_option)
)



In [184]:
job = bq.query(query = query)
job.result()

In [188]:
query = f'''
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts` AS
SELECT
    * EXCEPT(text_content, ml_translate_result, ml_translate_status),
    ml_translate_result.translations[0].translated_text
FROM ML.TRANSLATE(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.CLOUD_TRANSLATE`,
    (SELECT *, description as text_content
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_topMensProducts`),
    STRUCT('TRANSLATE_TEXT' AS translate_mode, 'es' AS target_language_code)
)
'''
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_topMensProducts` AS
SELECT
    * EXCEPT(text_content, ml_translate_result, ml_translate_status),
    ml_translate_result.translations[0].translated_text
FROM ML.TRANSLATE(
    MODEL `statmike-mlops-349915.bqml_2024.CLOUD_TRANSLATE`,
    (SELECT *, description as text_content
        FROM `statmike-mlops-349915.bqml_2024.thelook_topMensProducts`),
    STRUCT('TRANSLATE_TEXT' AS translate_mode, 'es' AS target_language_code)
)



In [189]:
job = bq.query(query = query)
job.result()

---
## Prepare Data

In [227]:
query = f'''
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_raw` AS (
    SELECT
        DATE(o.created_at) as order_date,
        o.order_id as order_id,
        o.user_id AS customer_id,
        oi.sale_price as unit_price,
        oi.product_id as product_id,
        oi.sale_price as revenue,
        u.age as age,
        u.country as country,
        u.gender as gender
    FROM `{BQ_SOURCE_DATASET}.orders` as o
    JOIN `{BQ_SOURCE_DATASET}.order_items` as oi ON o.user_id = oi.user_id
    JOIN `{BQ_SOURCE_DATASET}.users` as u ON o.user_id = u.id
    #WHERE DATE(o.created_at) < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
);
'''

In [228]:
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_raw` AS (
    SELECT
        DATE(o.created_at) as order_date,
        o.order_id as order_id,
        o.user_id AS customer_id,
        oi.sale_price as unit_price,
        oi.product_id as product_id,
        oi.sale_price as revenue,
        u.age as age,
        u.country as country,
        u.gender as gender
    FROM `bigquery-public-data.thelook_ecommerce.orders` as o
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` as oi ON o.user_id = oi.user_id
    JOIN `bigquery-public-data.thelook_ecommerce.users` as u ON o.user_id = u.id
    #WHERE DATE(o.created_at) < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
);



In [229]:
job = bq.query(query = query)
job.result()

In [230]:
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features` AS
WITH
    get_rfm AS (
        SELECT
            customer_id, age, gender, country,
            DATE_DIFF(DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH), MAX(order_date), DAY) AS recency,
            COUNT(order_id) AS frequency,
            SUM(revenue) AS monetary,
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_raw`
        WHERE order_date < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
        GROUP BY customer_id, age, gender, country
    ),
    get_quartiles AS (
        SELECT
            customer_id,
            NTILE(4) OVER (ORDER BY recency ASC) AS recency_quartile,
            NTILE(4) OVER (ORDER BY frequency DESC) AS frequency_quartile,
            NTILE(4) OVER (ORDER BY monetary DESC) AS monetary_quartile,
        FROM get_rfm
    ),
    get_rfm_score AS (
        SELECT
            customer_id,
            recency_quartile + frequency_quartile + monetary_quartile AS rfm_score
        FROM get_quartiles
    )
SELECT
    CAST(rfm.customer_id AS STRING) AS customer_id,
    rfm.age,
    rfm.gender,
    rfm.country,
    rfm.recency,
    rfm.frequency,
    rfm.monetary,
    q.recency_quartile,
    q.frequency_quartile,
    q.monetary_quartile,
    rfms.rfm_score
FROM get_rfm as rfm
LEFT JOIN get_quartiles as q ON rfm.customer_id = q.customer_id
LEFT JOIN get_rfm_score as rfms ON rfm.customer_id = rfms.customer_id
"""

In [231]:
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_features` AS
WITH
    get_rfm AS (
        SELECT
            customer_id, age, gender, country,
            DATE_DIFF(DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH), MAX(order_date), DAY) AS recency,
            COUNT(order_id) AS frequency,
            SUM(revenue) AS monetary,
        FROM `statmike-mlops-349915.bqml_2024.thelook_raw`
        WHERE order_date < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
        GROUP BY customer_id, age, gender, country
    ),
    get_quartiles AS (
        SELECT
            customer_id,
            NTILE(4) OVER (ORDER BY recency ASC) AS recency_quartile,
            NTILE(4) OVER (ORDER BY frequency DESC) AS frequency_quartile,
            NTILE(4) OVER (ORDER BY monetary DESC) AS monetary_quartile,
        FROM get_rfm
    ),
    get_rfm_score AS (
        SELECT
            customer_id,
            recency_quartile + frequency_quartile + monetary_quartile AS rfm_score
        FROM 

In [232]:
job = bq.query(query = query)
job.result()

---
## Customer Segmentation

In [23]:
query = f'''
CREATE MODEL IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`
OPTIONS (
    # model specs
        model_type = 'KMEANS',
        num_clusters = HPARAM_RANGE(2, 20),
        kmeans_init_method = 'KMEANS++',
        distance_type = 'EUCLIDEAN', 
        standardize_features = TRUE,
        
    # training specs
        max_iterations = 25,
        early_stop = TRUE,
        min_rel_progress = 0.005,
        
    # hyperparameter specs
        hparam_tuning_algorithm = 'VIZIER_DEFAULT',
        hparam_tuning_objectives = ['davies_bouldin_index'],
        num_trials = 12,
        max_parallel_trials = 3
    ) AS
SELECT * EXCEPT(customer_id)
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features`
'''

In [24]:
print(query)


CREATE MODEL IF NOT EXISTS `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`
OPTIONS (
        # model specs
        model_type = 'KMEANS',
        num_clusters = HPARAM_RANGE(2, 20),
        kmeans_init_method = 'KMEANS++',
        distance_type = 'EUCLIDEAN', 
        standardize_features = TRUE,
        
        # training specs
        max_iterations = 25,
        early_stop = TRUE,
        min_rel_progress = 0.005,
        
        # hyperparameter specs
        hparam_tuning_algorithm = 'VIZIER_DEFAULT',
        hparam_tuning_objectives = ['davies_bouldin_index'],
        num_trials = 12,
        max_parallel_trials = 3
    ) AS
SELECT * EXCEPT(customer_id)
FROM `statmike-mlops-349915.bqml_2024.thelook_features`



In [25]:
job = bq.query(query = query)
job.result()

In [29]:
query = f'''
SELECT *
FROM ML.PREDICT (
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`,
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features` LIMIT 5)
)
'''

In [30]:
print(query)


SELECT *
FROM ML.PREDICT (
    MODEL `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`,
    (SELECT * FROM `statmike-mlops-349915.bqml_2024.thelook_features` LIMIT 5)
)



In [31]:
job = bq.query(query = query)
job.result()
job.to_dataframe()

,trial_id,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,customer_id,age,gender,country,recency,frequency,monetary,recency_quartile,frequency_quartile,monetary_quartile,rfm_score
0,9,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.622465202475...",96858,25,M,United States,256,15,839.730002,2,1,1,4
1,9,5,"[{'CENTROID_ID': 5, 'DISTANCE': 1.276421759796...",12924,43,M,China,256,6,418.979996,2,1,1,4
2,9,5,"[{'CENTROID_ID': 5, 'DISTANCE': 2.007600684805...",34914,66,M,United Kingdom,256,8,567.260008,2,1,1,4
3,9,5,"[{'CENTROID_ID': 5, 'DISTANCE': 1.926941909655...",61152,29,F,Brasil,256,6,747.160000,2,1,1,4
4,9,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.350996319935...",5150,45,F,Australia,256,18,980.790018,2,1,1,4


In [225]:
query = f'''
SELECT *
FROM (
    SELECT *
    FROM ML.DETECT_ANOMALIES (
        MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`,
        STRUCT(0.01 AS contamination),
        (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features`)
    )
)
WHERE is_anomaly = TRUE
ORDER BY normalized_distance DESC
'''
print(query)


SELECT *
FROM (
    SELECT *
    FROM ML.DETECT_ANOMALIES (
        MODEL `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`,
        STRUCT(0.01 AS contamination),
        (SELECT * FROM `statmike-mlops-349915.bqml_2024.thelook_features`)
    )
)
WHERE is_anomaly = TRUE
ORDER BY normalized_distance DESC



In [226]:
job = bq.query(query = query)
job.result()
job.to_dataframe()

,trial_id,is_anomaly,normalized_distance,CENTROID_ID,customer_id,age,gender,country,recency,frequency,monetary,recency_quartile,frequency_quartile,monetary_quartile,rfm_score
0,9,True,3.475489,3,69909,66,M,Germany,168,40,6026.319988,2,1,1,4
1,9,True,3.322007,3,65171,23,F,France,153,36,5924.119995,2,1,1,4
2,9,True,2.932821,3,71161,35,M,Belgium,220,36,5500.040016,2,1,1,4
3,9,True,2.849891,3,88447,38,M,United States,123,40,5326.719986,2,1,1,4
4,9,True,2.802084,3,60215,39,F,China,83,44,5129.719994,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,9,True,1.411050,1,3084,40,F,Germany,1108,10,966.799971,4,1,1,6
430,9,True,1.410962,4,48406,27,M,Brasil,646,9,1210.080002,4,1,1,6
431,9,True,1.410729,8,47029,22,M,United Kingdom,1229,2,125.950001,4,2,2,9
432,9,True,1.410650,1,27572,29,M,China,1363,4,175.410000,4,2,2,8


In [213]:
query = f'''
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_segmentSummary` AS
WITH
    SEGMENTS AS (
        SELECT * EXCEPT(TRIAL_ID, NEAREST_CENTROIDS_DISTANCE)
        FROM ML.PREDICT (
            MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features_kmeans`,
            (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_features`)
        )
    )
SELECT
    CENTROID_ID,
    100 * (count(*) / sum(count(*)) OVER()) AS pct,
    AVG(age) AS avg_age,
    APPROX_TOP_COUNT(gender, 1)[OFFSET(0)].value AS mode_gender,
    APPROX_TOP_COUNT(country, 1)[OFFSET(0)].value AS first_country,
    APPROX_TOP_COUNT(country, 2)[OFFSET(1)].value AS second_country,
    APPROX_TOP_COUNT(country, 3)[OFFSET(2)].value AS third_country,
    APPROX_TOP_COUNT(country, 4)[OFFSET(3)].value AS fourth_country,
    AVG(recency) AS avg_recency,
    AVG(frequency) AS avg_frequency,
    AVG(monetary) AS avg_monetary,
    APPROX_TOP_COUNT(recency_quartile, 1)[OFFSET(0)].value AS mode_recency_quartile,
    APPROX_TOP_COUNT(frequency_quartile, 1)[OFFSET(0)].value AS mode_frequency_quartile,
    APPROX_TOP_COUNT(monetary_quartile, 1)[OFFSET(0)].value AS mode_monetary_quartile,
    AVG(rfm_score) as avg_rfm_score
FROM SEGMENTS
GROUP BY CENTROID_ID
ORDER BY CENTROID_ID
'''

In [214]:
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_segmentSummary` AS
WITH
    SEGMENTS AS (
        SELECT * EXCEPT(TRIAL_ID, NEAREST_CENTROIDS_DISTANCE)
        FROM ML.PREDICT (
            MODEL `statmike-mlops-349915.bqml_2024.thelook_features_kmeans`,
            (SELECT * FROM `statmike-mlops-349915.bqml_2024.thelook_features`)
        )
    )
SELECT
    CENTROID_ID,
    100 * (count(*) / sum(count(*)) OVER()) AS pct,
    AVG(age) AS avg_age,
    APPROX_TOP_COUNT(gender, 1)[OFFSET(0)].value AS mode_gender,
    APPROX_TOP_COUNT(country, 1)[OFFSET(0)].value AS first_country,
    APPROX_TOP_COUNT(country, 2)[OFFSET(1)].value AS second_country,
    APPROX_TOP_COUNT(country, 3)[OFFSET(2)].value AS third_country,
    APPROX_TOP_COUNT(country, 4)[OFFSET(3)].value AS fourth_country,
    AVG(recency) AS avg_recency,
    AVG(frequency) AS avg_frequency,
    AVG(monetary) AS avg_monetary,
    APPROX_TOP_COUNT(recency_quartile, 1)[OFFSET(0)].value AS mode_recency_quartile,
  

In [215]:
job = bq.query(query = query)
job.result()

In [216]:
query = f'''
WITH
    SHOTS AS (
        SELECT
            CONCAT(
                'Cluster ', CAST(CENTROID_ID AS STRING), ', ',
                CAST(ROUND(pct, 2) AS STRING), ' percent of customers, ',
                CAST(ROUND(avg_age, 2) AS STRING), ' average age of customers, ',
                CAST(ROUND(avg_recency, 2) AS STRING), ' average days since order, ',
                CAST(ROUND(avg_frequency, 2) AS STRING), ' average total orders, ', 
                CAST(ROUND(avg_monetary, 2) AS STRING), ' average total spend, ',
                CAST(mode_recency_quartile AS STRING), ' most common quartile for days since order, ',
                CAST(mode_frequency_quartile AS STRING), ' most common quartile for total orders, ',
                CAST(mode_monetary_quartile AS STRING), ' most common quartile for total spend, ',
                'most common countries: ', first_country, ', ', second_country, ', ', third_country, ', ', fourth_country,
                '.'
            ) as shot
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_segmentSummary`
    ),
    PROMPT AS (
        SELECT
            CONCAT(
                'The customers for your retail store have be grouped in clusters. ',
                'Create a description for each of these clusters:',
                '\\n\\n',
                STRING_AGG(shot, '\\n')
            ) as prompt
        FROM SHOTS
    )
SELECT *
FROM PROMPT
'''

In [217]:
print(query)


WITH
    SHOTS AS (
        SELECT
            CONCAT(
                'Cluster ', CAST(CENTROID_ID AS STRING), ', ',
                CAST(ROUND(pct, 2) AS STRING), ' percent of customers, ',
                CAST(ROUND(avg_age, 2) AS STRING), ' average age of customers, ',
                CAST(ROUND(avg_recency, 2) AS STRING), ' average days since order, ',
                CAST(ROUND(avg_frequency, 2) AS STRING), ' average total orders, ', 
                CAST(ROUND(avg_monetary, 2) AS STRING), ' average total spend, ',
                CAST(mode_recency_quartile AS STRING), ' most common quartile for days since order, ',
                CAST(mode_frequency_quartile AS STRING), ' most common quartile for total orders, ',
                CAST(mode_monetary_quartile AS STRING), ' most common quartile for total spend, ',
                'most common countries: ', first_country, ', ', second_country, ', ', third_country, ', ', fourth_country,
                '.'
            ) as shot
    

In [218]:
job = bq.query(query = query)
job.result()
result = job.to_dataframe()
result

,prompt
0,The customers for your retail store have be gr...


In [219]:
print(result['prompt'].iloc[0])

The customers for your retail store have be grouped in clusters. Create a description for each of these clusters:

Cluster 1, 13.65 percent of customers, 42.58 average age of customers, 617.76 average days since order, 5.39 average total orders, 337.03 average total spend, 4 most common quartile for days since order, 1 most common quartile for total orders, 1 most common quartile for total spend, most common countries: China, United States, Brasil, South Korea.
Cluster 2, 13.77 percent of customers, 27.06 average age of customers, 187.85 average days since order, 2.93 average total orders, 164.61 average total spend, 1 most common quartile for days since order, 2 most common quartile for total orders, 2 most common quartile for total spend, most common countries: China, United States, Brasil, South Korea.
Cluster 3, 1.11 percent of customers, 42.07 average age of customers, 200.99 average days since order, 25.24 average total orders, 2253.14 average total spend, 1 most common quartile 

In [220]:
query = f'''
WITH
    SHOTS AS (
        SELECT
            CONCAT(
                'Cluster ', CAST(CENTROID_ID AS STRING), ', ',
                CAST(ROUND(pct, 2) AS STRING), ' percent of customers, ',
                CAST(ROUND(avg_age, 2) AS STRING), ' average age of customers, ',
                CAST(ROUND(avg_recency, 2) AS STRING), ' average days since order, ',
                CAST(ROUND(avg_frequency, 2) AS STRING), ' average total orders, ', 
                CAST(ROUND(avg_monetary, 2) AS STRING), ' average total spend, ',
                CAST(mode_recency_quartile AS STRING), ' most common quartile for days since order, ',
                CAST(mode_frequency_quartile AS STRING), ' most common quartile for total orders, ',
                CAST(mode_monetary_quartile AS STRING), ' most common quartile for total spend, ',
                'most common countries: ', first_country, ', ', second_country, ', ', third_country, ', ', fourth_country,
                '.'
            ) as shot
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_segmentSummary`
    ),
    PROMPT AS (
        SELECT
            CONCAT(
                'The customers for your retail store have be grouped in clusters. ',
                'Create a brief description for each of these clusters:',
                '\\n\\n',
                STRING_AGG(shot, '\\n')
            ) as prompt
        FROM SHOTS
    )
SELECT
    ml_generate_text_llm_result as response
FROM ML.GENERATE_TEXT(
    MODEL `{BQ_PROJECT}.{BQ_DATASET}.VERTEX_LLM`,
    (SELECT * FROM PROMPT),
    STRUCT(1000 AS max_output_tokens, TRUE AS flatten_json_output)
)
'''
print(query)


WITH
    SHOTS AS (
        SELECT
            CONCAT(
                'Cluster ', CAST(CENTROID_ID AS STRING), ', ',
                CAST(ROUND(pct, 2) AS STRING), ' percent of customers, ',
                CAST(ROUND(avg_age, 2) AS STRING), ' average age of customers, ',
                CAST(ROUND(avg_recency, 2) AS STRING), ' average days since order, ',
                CAST(ROUND(avg_frequency, 2) AS STRING), ' average total orders, ', 
                CAST(ROUND(avg_monetary, 2) AS STRING), ' average total spend, ',
                CAST(mode_recency_quartile AS STRING), ' most common quartile for days since order, ',
                CAST(mode_frequency_quartile AS STRING), ' most common quartile for total orders, ',
                CAST(mode_monetary_quartile AS STRING), ' most common quartile for total spend, ',
                'most common countries: ', first_country, ', ', second_country, ', ', third_country, ', ', fourth_country,
                '.'
            ) as shot
    

In [221]:
job = bq.query(query = query)
job.result()
result = job.to_dataframe()
result

,response
0,**Cluster 1: Loyal Customers**\n- Middle-aged...


In [222]:
print(result['response'].iloc[0])

 **Cluster 1: Loyal Customers**
- Middle-aged customers (average age: 42.58)
- Long-term customers (average days since order: 617.76)
- High-value customers (average total spend: 337.03)
- Most common countries: China, United States, Brazil, South Korea

**Cluster 2: Frequent Buyers**
- Younger customers (average age: 27.06)
- Frequent buyers (average total orders: 2.93)
- Moderate spenders (average total spend: 164.61)
- Most common countries: China, United States, Brazil, South Korea

**Cluster 3: Big Spenders**
- Middle-aged customers (average age: 42.07)
- High-frequency buyers (average total orders: 25.24)
- Highest spenders (average total spend: 2253.14)
- Most common countries: China, United States, Brazil, South Korea

**Cluster 4: Occasional Buyers**
- Middle-aged customers (average age: 40.45)
- Occasional buyers (average total orders: 16.76)
- High spenders (average total spend: 1014.62)
- Most common countries: China, United States, Brazil, South Korea

**Cluster 5: Steady 

---
## Forecasting With Core Customers

In [11]:
query = f'''
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_forecastRaw` AS (
WITH
    RAW AS (
        SELECT
            DATE(o.created_at) as order_date,
            o.order_id as order_id,
            o.user_id AS customer_id,
            oi.sale_price as unit_price,
            oi.product_id as product_id,
            oi.sale_price as revenue,
            u.age as age,
            u.country as country,
            u.gender as gender
        FROM `{BQ_SOURCE_DATASET}.orders` as o
        JOIN `{BQ_SOURCE_DATASET}.order_items` as oi ON o.user_id = oi.user_id
        JOIN `{BQ_SOURCE_DATASET}.users` as u ON o.user_id = u.id
    ),
    PREP AS (
    
    ),
    SEGMENT AS (
    
    )
SELECT order_date, cluster, revenue
FROM SEGMENT
ORDER BY order_date, cluster
;
'''

In [12]:
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_2024.thelook_raw` AS (
    SELECT
        DATE(o.created_at) as order_date,
        o.order_id as order_id,
        o.user_id AS customer_id,
        oi.sale_price as unit_price,
        oi.product_id as product_id,
        oi.sale_price as revenue,
        u.age as age,
        u.country as country,
        u.gender as gender
    FROM `bigquery-public-data.thelook_ecommerce.orders` as o
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` as oi ON o.user_id = oi.user_id
    JOIN `bigquery-public-data.thelook_ecommerce.users` as u ON o.user_id = u.id
    WHERE DATE(o.created_at) < DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
);



In [13]:
job = bq.query(query = query)
job.result()

In [ ]:
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_arimaplus`
OPTIONS
  (model_type = 'ARIMA_PLUS',
   time_series_timestamp_col = '{TIME_COLUMN}',
   time_series_data_col = '{TARGET_COLUMN}',
   time_series_id_col = '{SERIES_COLUMN}',
   data_frequency = '{FORECAST_GRANULARITY}',
   auto_arima_max_order = 5,
   holiday_region = ['GLOBAL', 'US'],
   horizon = {FORECAST_HORIZON_LENGTH} + {FORECAST_TEST_LENGTH}
  ) AS
SELECT 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_PREFIX}_raw`
WHERE {SPLIT_COLUMN} in ('TRAIN','VALIDATE')
"""
job = bq.query(query)
job.result()
(job.ended-job.started).total_seconds()

---
## Early Categorization of New Customers